# 🆔 IDキャッシュ生成ツール
既存の `database.csv` (JRA) および `database_nar.csv` (NAR) をスキャンし、正しく取得されているレースIDを抽出してキャッシュファイルを作成します。
**これを実行することで、次回のスクレイピング時にこれら全てのレースをスキップ（高速化）できます。**

In [ ]:
# Google Driveをマウントする場合のみ実行してください
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 設定
DATA_DIR = '/content/drive/MyDrive/dai-keiba/data/raw' # データがあるフォルダ
JRA_CSV = 'database.csv'
JRA_ID_CSV = 'race_ids.csv'
NAR_CSV = 'database_nar.csv'
NAR_ID_CSV = 'race_ids_nar.csv'


In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm

def check_integrity(df_chunk):
    # 必須カラムチェック
    CRITICAL_COLS = ['race_id', 'horse_id', 'horse_name', 'jockey', 'date', 'rank']
    if not all(col in df_chunk.columns for col in CRITICAL_COLS):
        return False
    # 欠損値チェック
    if df_chunk[CRITICAL_COLS].isnull().any().any():
        return False
    return True

def generate_id_cache(target_csv, id_csv, name='JRA'):
    csv_path = os.path.join(DATA_DIR, target_csv)
    id_path = os.path.join(DATA_DIR, id_csv)
    
    if not os.path.exists(csv_path):
        print(f"⚠️ {name}: {csv_path} が見つかりません。スキップします。")
        return
        
    print(f"🔍 {name}: {csv_path} をスキャン中...")
    try:
        # Chunk読み込み等はせず、シンプルに実装（Colabのメモリを信じる）
        df = pd.read_csv(csv_path, dtype=str)
        
        if 'race_id' not in df.columns:
            print(f"❌ {name}: race_idカラムがありません。")
            return
            
        valid_ids = []
        groups = df.groupby('race_id')
        
        for rid, group in tqdm(groups, desc=f'{name} Validating'):
            if check_integrity(group):
                valid_ids.append(rid)
        
        print(f"✨ {name}: {len(valid_ids)}件の有効なレースIDが見つかりました。")
        
        if valid_ids:
            pd.DataFrame({'race_id': valid_ids}).to_csv(id_path, index=False)
            print(f"✅ {id_path} に保存しました。")
        else:
            print(f"⚠️ 有効なレースが1件もありませんでした。")

    except Exception as e:
        print(f"❌ {name} 処理エラー: {e}")


In [ ]:
# 実行
if os.path.exists(DATA_DIR):
    generate_id_cache(JRA_CSV, JRA_ID_CSV, name='JRA')
    print('-'*30)
    generate_id_cache(NAR_CSV, NAR_ID_CSV, name='NAR')
else:
    print(f"❌ ディレクトリが見つかりません: {DATA_DIR}")
